In [ ]:
import pandas as pd
import glob as glob
import os
import numpy as np

Loading measurement datafiles

In [ ]:
#micromanipulator measurement
paths = glob.glob(os.path.join("E:/Turku_collab/lunagel_50s/remodelling", "**","summary_ID_level.csv"),recursive=True)
print(*paths, sep='\n')

#cell info
cell_info = (pd.read_csv("E:/Turku_collab/lunagel_50s/remodelling/cell_info.csv")).drop(columns=['Unnamed: 0'])
cell_info['day'] = cell_info['day'].astype(str)
cell_info['experiment'] = cell_info['experiment'].astype(str)

Parsing data

In [ ]:
df=[]

fov_um = 2048*0.325

def parser(i):
    data = pd.read_csv(i)
    data['irradiation'] = (i.split('/')[2].split('_')[1])       
    data['incubation_set'] = i.split('/')[3].split('\\')[1]
    data['incubation'] = i.split('/')[3].split('\\')[2]
    data['experiment'] = i.split('/')[3].split('\\')[3].split('_')[0]
    data['cell/control'] = i.split('/')[3].split('\\')[4].split('_')[-1]
    data['day'] = data['day'].astype(str)
    max_location = data['location'].unique().max()
    
    data['location_inverse'] = max_location + 1 - data['location']

    for j, location in enumerate(data['location'].unique()):
        # Use .loc to properly assign values to the dataframe
        data.loc[data['location'] == location, 'gradient'] = data['y'] + location*fov_um
        data.loc[data['location'] == location, 'gradient_inverse'] = data['y'] + (max_location-location)*fov_um
    return data
  
df = pd.concat([parser(i) for i in paths], ignore_index=True)

Direction of the gradient

In [ ]:
df['gradient_direction'] = np.where(
    ((df['sample'] == "MDAMB231") & (df['day'] == "241107")) | 
    (df['day'].isin(["241121","241122", "241125","241126","240830"])),
    "stiff_to_soft",
    "soft_to_stiff")

Parameters for selection of close-by beads


In [ ]:
time = 180 #min
speed = 0.5 # or 0.6µm/min
pixel_size = 0.325 #µm
distance = time*speed/pixel_size
print(distance)

Assigning to the relevant measurements if cells closeby of faraway

In [ ]:
grouped_all = df.groupby(['day', 'experiment', 'holder', 'location'])
grouped_cell = cell_info.groupby(['day', 'experiment', 'holder', 'location'])

# Add a new column to the original df for classification
df['cell_rel_distance'] = 'unclassified'  # Default value for debugging

# Initialize lists to store faraway and closeby beads
closeby = []
faraway = []

# Initialize counters for debugging
closeby_count = 0
faraway_count = 0

# Track unmatched keys
unmatched_keys = set(grouped_all.groups.keys())

# Iterate through the groups
for key_all, j_all in grouped_all:
    matched = False  # Flag to check if a matching key was found
    for key_cell, j_cell in grouped_cell:
        if key_all == key_cell:
            matched = True  # Mark as matched
            unmatched_keys.discard(key_all)  # Remove from unmatched keys
            # print(f"Processing group: {key_all}")
            for index_all, row_all in j_all.iterrows():
                bead_classified = False
                for index_cell, row_cell in j_cell.iterrows():
                    # Check if the bead is close to any cell
                    if ((row_cell['x'] - distance) < row_all['x'] < (row_cell['x'] + distance)) and \
                       ((row_cell['y'] - distance) < row_all['y'] < (row_cell['y'] + distance)):
                        # Assign 'close' and append to closeby list
                        df.at[index_all, 'cell_rel_distance'] = 'close'
                        closeby.append(row_all)
                        bead_classified = True
                        closeby_count += 1
                        break
                if not bead_classified:
                    # Assign 'far' and append to faraway list
                    df.at[index_all, 'cell_rel_distance'] = 'far'
                    faraway.append(row_all)
                    faraway_count += 1
    if not matched:
        print(f"No matching key_cell for key_all: {key_all}")

# Convert closeby and faraway lists into DataFrames
final_closeby = pd.DataFrame(closeby)
final_faraway = pd.DataFrame(faraway)

print(df['cell_rel_distance'].value_counts())

Saving datafiles

In [ ]:
saving_path = 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/data'

df.to_csv(saving_path+'/all.csv')  #all data
final_closeby.to_csv(saving_path+'/closeby_beads.csv') #only closeby beads
final_faraway.to_csv(saving_path+'/faraway_beads.csv') #only faraway beads